In [1]:
import torch

In [3]:
data = torch.load('data/test-sample.pt')

In [4]:
for k, v in data.items():
    print(k, v.dtype, v.size())

sequence torch.float32 torch.Size([131072, 4])
target torch.float32 torch.Size([896, 5313])


In [14]:
2**17 / 2**7

1024.0

In [11]:
tf_gammas = torch.load('enformer_pytorch/precomputed/tf_gammas.pt')

In [12]:
tf_gammas.shape

torch.Size([3071, 32])